In [ ]:
!py -3.9 -m pip install folium

In [32]:
import pandas as pd
from IPython.display import display, HTML
# "<style>.container { width:100% !important; }</style>"
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 100)

In [ ]:
import random
import folium
import requests
from folium.plugins import MarkerCluster

"""
useful:
    https://maja0108.github.io/ip-post/
    https://msticpy.readthedocs.io/en/latest/visualization/FoliumMap.html
    https://towardsdatascience.com/creating-a-simple-map-with-folium-and-python-4c083abfff94
    https://python-visualization.github.io/folium/modules.html
    https://pypi.org/project/folium/
    https://www.python-graph-gallery.com/312-add-markers-on-folium-map
    https://ipinfo.io/
    https://gis.stackexchange.com/questions/437941/draw-a-line-between-pairs-of-points-with-calculating-the-distance-using-folium
    
todo:
    -markers with lines
"""


def ipinfo(ip):
    """request for ipinfo
    example:
    {
      "ip": "1.2.3.4",
      "city": "Brisbane",
      "region": "Queensland",
      "country": "AU",
      "loc": "-27.4820,153.0136",
      "postal": "4101",
      "timezone": "Australia/Brisbane",
      "readme": "https://ipinfo.io/missingauth"
    }
    """
    base_url = 'https://ipinfo.io/{}'
    url = base_url.format(ip)
    response = requests.get(url)
    return response.json()


def ipinfo_to_geolocation(data):
    """parse ipinfo json response and retur geolocation"""
    loc = data['loc']
    lat, lon = [float(item) for item in loc.split(',')]
    return lat, lon


# ******* read list of ips *******
# Recently Reported IPs from: https://www.abuseipdb.com/ 
ips = ['190.117.221.44', '119.96.175.9', '212.192.219.138', '190.188.229.150', '47.91.28.226', '157.245.51.40', '1.58.223.117', '183.99.2.239', '165.227.118.71', '58.34.189.28', '41.111.213.190', '40.127.173.225', '182.34.213.212', '142.93.116.249', '178.128.100.248', '14.171.211.221', '121.234.236.49', '120.48.15.186', '211.63.188.3', '187.149.142.97']
ips += ['128.90.162.149', '161.35.230.183', '106.56.250.2', '31.220.1.83', '134.122.52.34', '103.17.198.24', '163.44.252.65', '77.20.117.212', '165.227.110.188', '185.229.29.195', '86.105.27.142', '60.14.31.227', '61.177.172.19', '51.89.164.205', '84.201.177.10', '122.237.44.252', '119.187.147.110', '91.211.246.241', '13.233.136.87', '190.111.182.11']
ips = ips[:10]


# ******* get ips geolocation *******
ips_geo = []
for ip in ips:
    info = ipinfo(ip)
    lat, lon = ipinfo_to_geolocation(info)
    ips_geo.append((ip, lat, lon))
    
# ******* for content *******
ips_geo_pairs = []
first = ips_geo.pop(random.randrange(len(ips_geo)))
second = ips_geo.pop(random.randrange(len(ips_geo)))
ips_number = len(ips_geo)
for x in range(ips_number):
    right = ips_geo.pop(random.randrange(len(ips_geo)))
    # left = random.choice([first, second])
    left = random.choice([first])
    ips_geo_pairs.append((left, right))
    
# ******* map object *******
map_obj = folium.Map(location=[0, 0], tiles="OpenStreetMap", zoom_start=2)
marker_cluster = MarkerCluster().add_to(map_obj)

# ******* plot markers with polylines *******
if True:
    for (left, right) in ips_geo_pairs:
        left_ip, left_lat, left_lon = left
        right_ip, right_lat, right_lon = right
        folium.CircleMarker(location=[left_lat, left_lon], radius=5, fill_opacity=0.9).add_to(marker_cluster)
        folium.CircleMarker(location=[right_lat, right_lon], radius=5, fill_opacity=0.9).add_to(marker_cluster)
        folium.PolyLine([
            [left_lat, left_lon],
            [right_lat, right_lon]],
            color='red',
        ).add_to(marker_cluster)
        
# ******* plot markers only *******
if False:
    for ip, lat, lon in ips_geo:
        folium.CircleMarker(location=[lat, lon], radius=5, fill_opacity=0.9).add_to(marker_cluster)
        
# ******* display map *******
map_obj.save('ip_location.html')
display(map_obj)
